In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import lmfit
import numpy as np
from astropy.io import ascii
from astropy import units as u
from lmfit.models import PolynomialModel, GaussianModel
from lmfit import Model as mod
from lmfit import minimize, Parameters, report_fit
import ccdproc as ccdp
from pathlib import Path
import os
import glob
import shutil

In [2]:
os.makedirs('HIplots', exist_ok=True)
os.makedirs('HIgaussianplots', exist_ok=True)

In [2]:
# Plotting all HI
# Load all files matching pattern
file_list = sorted(glob.glob('HISpectra/*.txt'))  # Assumes files are in the same folder as this script
# Process each file
for filename in file_list:
    filename_base = os.path.splitext(os.path.basename(filename))[0]
    data = pd.read_csv(
        filename,
        skiprows=12,
        sep='\s+',
        header=None,
        engine='python',
        on_bad_lines='skip',  # skip lines with wrong number of columns
        nrows=595
    )
    # Name the columns explicitly
    data.columns = ['col1', 'col2']
    # Extract x and y
    x = data['col1']
    y = data['col2']
    # converting to numpy for the gooddata
    x = x.to_numpy()
    y = y.to_numpy()
    gooddata = np.where((x >= -100) & (x <= 100))
    x = x[gooddata]
    y = y[gooddata]
    plt.xlim([-100, 100])
    plt.ylim([-5, 70])
    plt.plot(x,y)
    plt.axhline(0, color='green')
    plt.xlabel('V_lsr(km/s)') # Creates a label for the x axis
    plt.ylabel('Tb(K)') # Creates a label for the y axis
    plt.title(f'HI; {filename_base}') # Creates a title for the plot
    #plt.legend()
    plt.tight_layout()
    plt.savefig(f"HIplots/{filename_base}.png")
    plt.close()
    
    region = np.where((x >= -10) & (x <= 15))[0]
    ### Gaussian Fits ###
    gauss_model = GaussianModel()
    params = gauss_model.guess(y[region], x=x[region])
    fit = gauss_model.fit(y[region], params, x=x[region])
    y_fit = gauss_model.eval(fit.params, x=x)

    ### Plotting Gaussian###
    plt.figure(figsize=(7, 5))
    plt.axhline(0, color='k')
    plt.plot(x, y, label='Data', color='grey')
    plt.plot(x, y_fit, '-', label='Gaussian', color='r')
    plt.xlim([-10, 15])
    plt.ylim([-5, 70])
    plt.xlabel('V_lsr(km/s)') # Creates a label for the x axis
    plt.ylabel('Tb(K)') # Creates a label for the y axis
    plt.title(f'Gaussian of HIData; {filename_base}') # Creates a title for the plot
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"HIgaussianplots/gaussian_fits_{filename_base}.png")
    plt.close()

print("All 37 files processed.")

C:\Users\dhami\miniforge3\envs\STARTastro\Lib\site-packages\uncertainties\core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
C:\Users\dhami\miniforge3\envs\STARTastro\Lib\site-packages\uncertainties\core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
C:\Users\dhami\miniforge3\envs\STARTastro\Lib\site-packages\uncertainties\core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
C:\Users\dhami\miniforge3\envs\STARTastro\Lib\site-packages\uncertainties\core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")
C:\Users\dhami\miniforge3\en

All 37 files processed.


C:\Users\dhami\miniforge3\envs\STARTastro\Lib\site-packages\uncertainties\core.py:1024: UserWarning: Using UFloat objects with std_dev==0 may give unexpected results.
  warn("Using UFloat objects with std_dev==0 may give unexpected results.")


In [ ]:
print(file_list)

In [9]:
### Measure RMS ###
# RMS red
file_list = sorted(glob.glob('HISpectra/*.txt'))  # Assumes files are in the same folder as this script
# Store results
rms_data = []
for filename in file_list:
    # Load your data
    data = pd.read_csv(
        filename,
        skiprows=12,
        sep='\s+',
        header=None,
        engine='python',
        on_bad_lines='skip',  # skip lines with wrong number of columns
        nrows=595
    )
    data.columns = ['col1', 'col2']
    # Extract x and y
    x = data['col1']
    y = data['col2']
    # converting to numpy for the gooddata
    x = x.to_numpy()
    y = y.to_numpy()
    gooddata1 = list(np.where((x >= -20) & (x <= -10))[0])
    gooddata2 = list(np.where((x >= 15) & (x <= 30))[0])
    gooddata = gooddata1 + gooddata2
    noise=np.std(y[gooddata]) * u.K
    noise_sub=np.std(y[gooddata]) * u.K
    # Save results
    rms_data.append({
        "filename": filename,
        "noise(K)": noise.value,
        "noise_sub(K)": noise_sub.value
    })
# Convert into pandas
df = pd.DataFrame(rms_data)
print(df)

                           filename  noise(K)  noise_sub(K)
0   HISpectra\158.375-19.250_HI.txt  0.930786      0.930786
1   HISpectra\158.500-19.250_HI.txt  0.961162      0.961162
2   HISpectra\158.500-19.375_HI.txt  0.972524      0.972524
3   HISpectra\158.625-19.375_HI.txt  1.018068      1.018068
4   HISpectra\158.750-19.375_HI.txt  1.076827      1.076827
5   HISpectra\158.750-19.500_HI.txt  1.040315      1.040315
6   HISpectra\158.875-19.500_HI.txt  1.022310      1.022310
7   HISpectra\158.875-19.625_HI.txt  0.956221      0.956221
8   HISpectra\159.000-19.625_HI.txt  0.925154      0.925154
9   HISpectra\159.000-19.750_HI.txt  0.840518      0.840518
10  HISpectra\159.125-19.750_HI.txt  0.827126      0.827126
11  HISpectra\159.125-19.875_HI.txt  0.755100      0.755100
12  HISpectra\159.250-19.875_HI.txt  0.767523      0.767523
13  HISpectra\159.250-20.000_HI.txt  0.754476      0.754476
14  HISpectra\159.375-20.000_HI.txt  0.760478      0.760478
15  HISpectra\159.500-20.000_HI.txt  0.7

In [21]:
# Computing the Integral
def integral(vel, ta, bounds, deltav):
    mask = (vel >= bounds[0]) & (vel <= bounds[1])
    deltav = vel[1]-vel[0]
    return np.sum(ta[mask])*deltav
# Set the path to your ASCII files
file_list = sorted(glob.glob('HISpectra/*.txt'))  # Assumes files are in the same folder as this script
bounds = [-10, 15]
# Process each file
for filename in file_list:
    data = pd.read_csv(
        filename,
        skiprows=12,
        sep='\s+',
        header=None,
        engine='python',
        on_bad_lines='skip',  # skip lines with wrong number of columns
        nrows=595
    )
    data.columns = ['col1', 'col2']
    # Extract x and y
    vel = data['col1']
    ta = data['col2']
    deltav = vel[1]-vel[0]
    # Compute integral
    result = integral(vel, ta, bounds, deltav)
    print(f"{filename}: Integral from {bounds[0]} to {bounds[1]} = {result}")

HISpectra\158.375-19.250_HI.txt: Integral from -10 to 15 = 601.2548100000096
HISpectra\158.500-19.250_HI.txt: Integral from -10 to 15 = 575.6960400000091
HISpectra\158.500-19.375_HI.txt: Integral from -10 to 15 = 593.7005700000094
HISpectra\158.625-19.375_HI.txt: Integral from -10 to 15 = 569.640780000009
HISpectra\158.750-19.375_HI.txt: Integral from -10 to 15 = 557.3651400000089
HISpectra\158.750-19.500_HI.txt: Integral from -10 to 15 = 585.4961700000093
HISpectra\158.875-19.500_HI.txt: Integral from -10 to 15 = 572.4052500000091
HISpectra\158.875-19.625_HI.txt: Integral from -10 to 15 = 594.9183300000094
HISpectra\159.000-19.625_HI.txt: Integral from -10 to 15 = 569.5453200000092
HISpectra\159.000-19.750_HI.txt: Integral from -10 to 15 = 592.2996300000094
HISpectra\159.125-19.750_HI.txt: Integral from -10 to 15 = 569.5195200000089
HISpectra\159.125-19.875_HI.txt: Integral from -10 to 15 = 552.0219600000088
HISpectra\159.250-19.875_HI.txt: Integral from -10 to 15 = 534.6069600000085


In [20]:
print(deltav)

1.2900000000000205
